<center><h2><b>Covid19 Vaccination Progress</b></h2></center>

<center><b>** Seychelles - Fresh Cases in May 2021 **</b></center> 

# **My Observations:**

1. In top 20 countries with highest total vaccinations - 
    * **7 Asian**, **7 European**, **3 North American**, **2 South American**, **1 African countries**.
2. In top 50 countries with highest total vaccinations - 
    * **23 European**, **13 Asian**, **7 South American**, **5 North America**, **2 African countries**.   
3. In countries with least fully vaccinated people per hundred we have -
 * **Namibia** in Africa - **0.05**
 * **Ukraine** in Europe - **0.00**
 * **Kyrgyzstan** in Asia - **0.01**
 * **Guatemala** in North America - **0.01**
 * **Suriname** in South America - **0.01**
   

# **Contents:**
Following plots are shown in this notebook using plotnine python package.
1. **Percentage yet to be vaccinated to achieve herd immunity in select countries USA, China, Brazil, India, UK.**
2. **Daily Vaccination progress in top nations from each continent.**
3. **Top 20 Countries Based on Total Vaccinations**
4. **Top 20 Countries Based on People Vacccinated**
5. **Top 20 Countries Based on People Fully Vacccinated - Stacked with Population**
6. **Top 20 Vaccinated Countries Based on People Fully Vacccinated Per Hundred**
7. **Total Number of Vaccinations for each Vaccine**
8. **Continent wise Count of Countries with Top 50 Most People Fully Vacccinated**
9. **Country with Least Fully Vacccinated People Per Hundred in each Continent**

In [ ]:
!pip install pycountry-convert

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pandas.api.types import CategoricalDtype
from plotnine import *
from plotnine.data import mpg
from plotnine import options as op
from plotnine import watermark as wm
import matplotlib.pyplot as plt
from PIL import Image
import missingno as msno
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import pycountry_convert as pc
import warnings
warnings.filterwarnings('ignore')
import collections

In [ ]:
vaccination_data = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv')
population_data = pd.read_csv('../input/population-by-country-2020/population_by_country_2020.csv')
vaccination_data.columns

In [ ]:
from PIL import Image
import requests
import urllib
from io import StringIO
def image_resize(url):
    img = Image.open(requests.get(url, stream=True).raw)
    img = img.resize((70,55),Image.ANTIALIAS)
    img = np.asarray(img)
    return img

# **Fresh Cases in Seychelles:**

In [ ]:
seychelles_vaccination = vaccination_data[(vaccination_data.country == 'Seychelles')]['people_fully_vaccinated'].max()
seychelles_vaccines = vaccination_data[(vaccination_data.country == 'Seychelles')]['vaccines'].values[0]
seychelles_population = population_data[(population_data['Country (or dependency)'] == 'Seychelles')]['Population (2020)']
seychelles_may_2021 = pd.DataFrame(columns=['Fully Vaccianted','Population','Fresh Cases'])
seychelles_may_2021['Fully Vaccianted'] = [seychelles_vaccination]
seychelles_may_2021['Population'] = [seychelles_population.values[0]]
seychelles_may_2021['Fresh Cases'] = [9423]
seychelles_may_2021 = seychelles_may_2021.T
seychelles_may_2021 = seychelles_may_2021.reset_index()
seychelles_may_2021.columns = ['indicator','value']

In [ ]:
op.figure_size=(9, 4)
p = (ggplot(seychelles_may_2021)         
 + aes(y='value', x='indicator',fill='indicator') 
 + geom_text(x=1, y = 59676.0, label='As of May 9th 2021, fresh cases in Seychelles surged to ~10k \n despite getting most of its adult population vaccinated.',size=10,nudge_y=0.25,nudge_x=0.9,color='white')
 + geom_col(size=3)
 + coord_flip()
 + labs(title = "Seychelles - Fresh Cases in May 2021")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_position='none',
    panel_grid = element_line(size = 0.01),
    legend_text = element_text(color = "white"),
    axis_title_x = None,
    axis_text_y = element_text(color = "white", size = 10),
    axis_text_x = element_text(color = "white", size = 10),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = p.draw()
sc = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/SC_70x55.png')
fig.figimage(sc, xo=275, yo=75, alpha=1, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=False)
fig.show()

As per [reports](https://www.cnbc.com/2021/05/13/seychelles-most-vaccinated-nation-on-earth-but-covid-19-has-surged.html) covid-19 has surged in Seychelles, an archipelago of 115 islands in the Indian Ocean. It is vaccinated with Sinopharm and Covishield and nearly 60% of its population is fully vaccinated. It has to be seen as to how it progresses in coming days. 

In [ ]:
rename_country = ["England" , "Wales" , "Scotland" , "Northern Ireland"]

for i in rename_country:
     vaccination_data["country"] = vaccination_data["country"].str.replace(i , "United Kingdom")

In [ ]:
def convert_df_with_continent(df):
    
    df['continent'] = ['']*df.shape[0]
    
    def continent_name_convert(name):
        if name == 'AF':
            return 'Africa'
        elif name == 'EU':
            return 'Europe'
        elif name == 'NA':
            return 'NA'
        elif name == 'SA':
            return 'SA'
        elif name == 'OC':
            return 'Oceania'
        elif name == 'AS':
            return 'Asia'

    for i in range(df.shape[0]):
        country = df['country'].iloc[i]
        try:
            country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
            continent_name = pc.country_alpha2_to_continent_code(country_code)
            continent_name_full = continent_name_convert(continent_name)
            df['continent'].iloc[i] = continent_name_full
    #         print(f'{country} - {continent_name}')
        except:
            if country == "Cote d'Ivoire":
                continent_name = 'AF'
                continent_name_full = continent_name_convert(continent_name)
                df['continent'].iloc[i] = continent_name_full
            elif country == 'Faeroe Islands':
                continent_name = 'EU'
                continent_name_full = continent_name_convert(continent_name)
                df['continent'].iloc[i] = continent_name_full
            elif country == 'Kosovo':
                continent_name = 'EU'
                continent_name_full = continent_name_convert(continent_name)
                df['continent'].iloc[i] = continent_name_full
            elif country == 'Saint Helena':
                continent_name = 'AF'
                continent_name_full = continent_name_convert(continent_name)
                df['continent'].iloc[i] = continent_name_full
            elif country == 'Curacao':
                continent_name = 'SA'
                continent_name_full = continent_name_convert(continent_name)  
                df['continent'].iloc[i] = continent_name_full
            elif country == 'Timor':
                continent_name = 'AS'
                continent_name_full = continent_name_convert(continent_name)
                df['continent'].iloc[i] = continent_name_full
            elif country == 'Democratic Republic of Congo':
                continent_name = 'AF'
                continent_name_full = continent_name_convert(continent_name)
                df['continent'].iloc[i] = continent_name_full
            else:
                print(f'{country} mistake detected')
        continue
    return df



In [ ]:
def population_agg(df):
    df['population'] = ['']*df.shape[0]
    for i in range(df.shape[0]):
        country = df['country'].iloc[i]
        try:
            population = population_data.loc[population_data['Country (or dependency)'] == country]['Population (2020)'].values[0]
            df['population'].iloc[i] = population
        except Exception as e:
            df['population'].iloc[i] = np.nan
            continue
    return df

In [ ]:
def stack_population_df_fully_vaccinated(df):
    df_new = pd.DataFrame(columns=['country','vaccination_status','total_vaccinations'])
    df_new['country'] = ['']*2*df.shape[0]
    df_new['vaccination_status'] = ['']*2*df.shape[0]
    df_new['total_vaccinations'] = ['']*2*df.shape[0]
    index = 0
    for i in range(df.shape[0]):
        df_new.iloc[index] = [df['country'].iloc[i],'vaccinated',df['people_fully_vaccinated'].iloc[i]]
        df_new.iloc[index+1] = [df['country'].iloc[i],'not_vaccinated',int(df['population'].iloc[i])-int(df['people_fully_vaccinated'].iloc[i])]
        index += 2
    return df_new

In [ ]:
def format_number(number):
    return ("{:,}".format(number))

# **Percentage of Vaccination yet to be done to achieve Herd Immunity**

In [ ]:
countries_list = ['United States','United Kingdom','India',"Israel",'Brazil']
herd_immunity = pd.DataFrame()
for i in range(vaccination_data.shape[0]):
    if vaccination_data['country'].iloc[i] in countries_list:
        herd_immunity = herd_immunity.append(vaccination_data.iloc[i])
        
herd_immunity_df = pd.DataFrame(herd_immunity.groupby('country')['people_fully_vaccinated_per_hundred'].max())
herd_immunity_df['herd_immune_percent'] = ['']*herd_immunity_df.shape[0] 
for i in range(herd_immunity_df.shape[0]):
    herd_immunity_df['herd_immune_percent'].iloc[i] = round(70 - herd_immunity_df['people_fully_vaccinated_per_hundred'].iloc[i],2)
herd_immunity_df = herd_immunity_df.reset_index()
herd_immunity_df = population_agg(herd_immunity_df)
for i in range(herd_immunity_df.shape[0]):
    herd_immunity_df['population'].iloc[i] = format_number(round(herd_immunity_df['population'].iloc[i] * (herd_immunity_df['herd_immune_percent'].iloc[i]/100)))

In [ ]:
op.figure_size=(7,5)
ax = (ggplot(herd_immunity_df)         
 + aes(x='country', y='herd_immune_percent',fill='country')    
 + geom_col(size=15)
 + geom_label(aes(label = 'population'),color = "white",label_padding= 0.25,size=11)
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Percent Yet to be Vaccinated to achieve Herd Immunity")
 + labs(y = "Percentage", x = "Country")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.01),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white", size=13,face='bold'),
    axis_text_x = element_text(color = "white", size = 13,angle=90,face='bold'),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10}),
    axis_title = element_text(color = "white", size = 14, hjust = 1,face = "bold"),
    plot_title = element_text(color = "white", face = "bold", size = 12, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()


image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/16_1.png')
ind = image_resize('https://raw.githubusercontent.com/google/region-flags/gh-pages/png/IN.png')
brz = image_resize('https://raw.githubusercontent.com/google/region-flags/gh-pages/png/BR.png')
isr = image_resize('https://raw.githubusercontent.com/google/region-flags/gh-pages/png/IL.png')
usa = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/US_70x55.png')
uk = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/GB_70x55.png')
fig.figimage(image, xo=0, yo=0, alpha=0.12, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=False)
fig.figimage(ind, xo=220, yo=560, alpha=1, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=False)
fig.figimage(brz, xo=115, yo=485, alpha=1, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=False)
fig.figimage(isr, xo=327, yo=265, alpha=1, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=False)
fig.figimage(uk, xo=429, yo=415, alpha=1, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=False)
fig.figimage(usa, xo=533, yo=340, alpha=1, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=False)
fig.show()

Herd Immunity is said to be achieved when **70-75%** of population is vaccinated. Here I wanted to visualise how much percentage of population is yet to be vaccinated to achieve that target and what in absolute value that would mean in terms of population of that country.

I have selected five countries United States, United Kingdom, Brazil, India, Israel

> 1. **India** has to vaccinate **68%** more of its population with **Brazil** closely following India with **63%** more to be vaccinated.
> 2. **USA** has to vaccinate 37% while **UK** has to vaccinate 44%.
> 3. **Israel** on the other hand has to vaccinate only around **~11%** more to achieve herd immunity.

# **Daily Vaccination Progress In Top Nations From Each Continent**

In [ ]:
vaccination_data = convert_df_with_continent(vaccination_data)
daily_vaccinations = vaccination_data[['country','total_vaccinations','continent']]
daily_vaccine_df = pd.DataFrame(daily_vaccinations.groupby('continent')['total_vaccinations'].max().sort_values(ascending = False))
daily_vaccine_df = daily_vaccine_df.reset_index()
daily_vaccine_df_continent = pd.DataFrame(columns=['country','date','daily_vaccinations'])
for i in range(daily_vaccine_df.shape[0]):
    df = vaccination_data[(vaccination_data.continent==daily_vaccine_df['continent'].iloc[i]) & (vaccination_data.total_vaccinations==daily_vaccine_df['total_vaccinations'].iloc[i])]
    if df['country'].iloc[0] != 'United Kingdom':
        daily_vaccine_df_continent = daily_vaccine_df_continent.append(vaccination_data[(vaccination_data.country==df['country'].iloc[0])])
daily_vaccine_df_continent = daily_vaccine_df_continent[['country','date','daily_vaccinations']]
daily_vaccine_df_continent = daily_vaccine_df_continent.dropna(how='any',axis=0)
daily_vaccine_df_continent['date']= pd.to_datetime(daily_vaccine_df_continent['date'])
daily_vaccine_df_continent['daily_vaccinations'] = daily_vaccine_df_continent['daily_vaccinations'].astype(int)
op.figure_size=(12, 12)
p = (ggplot(daily_vaccine_df_continent)         
 + aes(y='daily_vaccinations', x='date',color='country')    
 + geom_line(size=1.5) 
 + scale_fill_manual(values = ["#B21236","#723240","#FE8826","#FEB914","#2C9FA2","#002C2B"])
#  + facet_wrap('country')
 + labs(title = "Daily Vaccination In Top Nations From Each Continent")
 + labs(y = "Daily Vaccination", x = "Date")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.1),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white"),
    axis_text_x = element_text(color = "white", size = 10,angle=90),
    axis_text_y = element_text(color = "white", size = 10),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = p.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/14_1.png')

fig.figimage(image, xo=0, yo=0, alpha=0.2, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

A comparison of daily vaccinations from top nations by total vaccination count from each continent.Here we can see how US and China compare in terms of daily vaccinations.

# **Top 20 Vaccinated Countries by Total Vaccinations**

In [ ]:
top_20_countries_vaccinated = pd.DataFrame(vaccination_data.groupby("country")["total_vaccinations"].max().sort_values(ascending = False).head(20))
top_20_countries_vaccinated.reset_index(level=0,inplace=True)
top_20_countries_vaccinated = convert_df_with_continent(top_20_countries_vaccinated)
top_20_countries_vaccinated['number_labels'] = ['']*top_20_countries_vaccinated.shape[0]

for i in range(top_20_countries_vaccinated.shape[0]):
    top_20_countries_vaccinated['number_labels'].iloc[i] = format_number(top_20_countries_vaccinated['total_vaccinations'].iloc[i])
    
op.figure_size=(8,5)
ax = (ggplot(top_20_countries_vaccinated)         
 + aes(x='country', y='total_vaccinations',fill='continent')    
 + geom_col(size=20)
#  + geom_label(aes(label = 'number_labels'),ha='right',color = "white",label_padding= 0.25)
 + coord_flip()
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Top 20 Vaccinated Countries By Total Vaccinations")
 + labs(y = "Number of Vaccinations", x = "Country")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.3),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white"),
    axis_text_x = element_text(color = "white", size = 20),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10}),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 20, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/1_enlargev3.png')

fig.figimage(image, xo=0, yo=0, alpha=0.2, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

USA has the most total vaccinations count followed by China, India, UK. Here we have7 Asian, 7 European, 3 North American, 2 South American, 1 Afriacn countries.

# **Top 20 Countries Based on People Vaccinated**

In [ ]:
top_20_countries_people_vaccinated = pd.DataFrame(vaccination_data.groupby("country")["people_vaccinated"].max().sort_values(ascending = False).head(20))
top_20_countries_people_vaccinated.reset_index(level=0,inplace=True)
top_20_countries_people_vaccinated = convert_df_with_continent(top_20_countries_people_vaccinated)
top_20_countries_people_vaccinated['number_labels'] = ['']*top_20_countries_people_vaccinated.shape[0]

for i in range(top_20_countries_people_vaccinated.shape[0]):
    top_20_countries_people_vaccinated['number_labels'].iloc[i] = format_number(top_20_countries_people_vaccinated['people_vaccinated'].iloc[i])
    
op.figure_size=(8,10)
ax = (ggplot(top_20_countries_people_vaccinated)         
 + aes(x='country', y='people_vaccinated',fill='continent')    
 + geom_col(size=15)
 + geom_label(aes(label = 'number_labels'),ha='right',color = "white",label_padding= 0.25,size=11)
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Top 20 Vaccinated Countries by People Vaccinated")
 + labs(y = "Number of Vaccinations", x = "Country")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.3),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white", size=13,face='bold'),
    axis_text_x = element_text(color = "white", size = 13,angle=90,face='bold'),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10}),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/8_enlargev2.png')

fig.figimage(image, xo=0, yo=0, alpha=0.2, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

We can see here the total vaccinated people in top 20 countries. USA again tops the list with nearly 120 million people vaccinated. Followed by India, UK, Brazil.

Now lets see not vaccinated poplutation from each of these countries to get a better picture.

# **Top 20 Vaccinated Countries based on People Fully Vacccinated**

In [ ]:
top_20_countries_people_fully_vaccinated = pd.DataFrame(vaccination_data.groupby("country")["people_fully_vaccinated"].max().sort_values(ascending = False).head(20))
top_20_countries_people_fully_vaccinated.reset_index(level=0,inplace=True)
top_20_countries_people_fully_vaccinated = convert_df_with_continent(top_20_countries_people_fully_vaccinated)
top_20_countries_people_fully_vaccinated = population_agg(top_20_countries_people_fully_vaccinated)
top_20_countries_people_fully_vaccinated = stack_population_df_fully_vaccinated(top_20_countries_people_fully_vaccinated)


op.figure_size=(8,10)
ax = (ggplot(top_20_countries_people_fully_vaccinated)         
 + aes(x='country', y='total_vaccinations',fill='vaccination_status')    
 + geom_col(size=20)
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Top 20 Vaccinated Countries based on People Fully Vaccinated")
 + labs(y = "Number of Vaccinations", x = "Country")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.3),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white",size = 10,face = "bold"),
    axis_text_x = element_text(color = "white", size = 13,angle=90,face = "bold"),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10},face = "bold"),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/11_enlargev1.png')

fig.figimage(image, xo=0, yo=0, alpha=0.2, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

We can clearly see that all of these counties still have a lot of not vaccinated population.It is clearly evident from this plot specially for heavily populated county like India.

**Israel stands out as the country that has more vaccinated than not vaccinated.**

# **Top 20 Vaccinated Countries based on People Fully Vacccinated per Hundred**

In [ ]:
top_20_countries_pfvph = pd.DataFrame(vaccination_data.groupby("country")["people_fully_vaccinated_per_hundred"].max().sort_values(ascending = False).head(20))
top_20_countries_pfvph.reset_index(level=0,inplace=True)
top_20_countries_pfvph = convert_df_with_continent(top_20_countries_pfvph)

top_20_countries_pfvph['number_labels'] = ['']*top_20_countries_pfvph.shape[0]

for i in range(top_20_countries_pfvph.shape[0]):
    top_20_countries_pfvph['number_labels'].iloc[i] = format_number(top_20_countries_pfvph['people_fully_vaccinated_per_hundred'].iloc[i])
op.figure_size=(5,5)
ax = (ggplot(top_20_countries_pfvph)         
 + aes(x='country', y='people_fully_vaccinated_per_hundred',fill='continent')    
 + geom_col(size=13)
 + geom_label(aes(label = 'number_labels'),ha='right',color = "white",label_padding= 0.25)
 + coord_flip()
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Top 20 Vaccinated Countries by People Fully Vaccinated Per Hundred")
 + labs(y = "Ratio (in percent)", x = "Country")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.3),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white",size=11,face='bold'),
    axis_text_x = element_text(color = "white", size = 10),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10}),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/12.png')

fig.figimage(image, xo=0, yo=0, alpha=0.15, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

In top countries with people fully vaccinated per hundred we have 
   * **Gibraltar** - Europe
   * **Israel** - Asia
   * **Seychelles** - Africa
   * **Falkland Islands** - South America
   * **Cayman Islands** - North America

**Gibraltar** has vaccinated almost all of its adult population. <br>
More than half of the population is vaccinated in **Israel**, **Seychelles** and **Falkland Islands**.

# **Total Vaccinations of each Vaccine**

In [ ]:
vaccine_manufacture = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations_by_manufacturer.csv')

vaccine_counts = pd.DataFrame(vaccine_manufacture.groupby("vaccine")["total_vaccinations"].max().sort_values(ascending = False))
vaccine_counts.reset_index(level=0,inplace=True)

vaccine_counts['number_labels'] = ['']*vaccine_counts.shape[0]

for i in range(vaccine_counts.shape[0]):
    vaccine_counts['number_labels'].iloc[i] = format_number(vaccine_counts['total_vaccinations'].iloc[i])
    

In [ ]:
op.figure_size=(8,5)
ax = (ggplot(vaccine_counts)         
 + aes(x='vaccine', y='total_vaccinations')    
 + geom_col(size=15)
 + geom_label(aes(label = 'number_labels'),ha='center',color = "black",label_padding= 0.25)
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Vaccine Counts")
 + labs(y = "Total No. of Vaccinations", x = "Vaccine")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "blue"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.3),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white"),
    axis_text_x = element_text(color = "white", size = 10),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10}),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/4.png')

fig.figimage(image, xo=0, yo=0, alpha=0.2, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

The total number of vaccinations of each vaccine are:
   * **Pfizer/BioNTech** - 134.1 M
   * **Moderna** - 109 M
   * **Sinovac** - 13 M
   * **Oxford/AstraZeneca** - 6 M
   * **Johnson&Jonson** - 8.6 M 

# **Top 50 Highest Vaccinated Countries Count in each Continent**

In [ ]:
top_50_countries_people_fully_vaccinated = pd.DataFrame(vaccination_data.groupby("country")["people_fully_vaccinated"].max().sort_values(ascending = False).head(50))
top_50_countries_people_fully_vaccinated.reset_index(level=0,inplace=True)
top_50_countries_people_fully_vaccinated = convert_df_with_continent(top_50_countries_people_fully_vaccinated)
top50_countries_continent_count = pd.DataFrame(top_50_countries_people_fully_vaccinated.groupby('continent')['country'].apply(list))
top50_countries_continent_count['country_count'] = ['']*top50_countries_continent_count.shape[0]
top50_countries_continent_count.reset_index(level=0,inplace=True)

for i in range(top50_countries_continent_count.shape[0]):
    top50_countries_continent_count['country_count'].iloc[i] = len(top50_countries_continent_count['country'].iloc[i])
    if top50_countries_continent_count['continent'].iloc[i] == 'NA':
        top50_countries_continent_count['continent'].iloc[i] = 'North America'
    elif top50_countries_continent_count['continent'].iloc[i] == 'SA':
        top50_countries_continent_count['continent'].iloc[i] = 'South America'

op.figure_size=(8,5)
ax = (ggplot(top50_countries_continent_count)         
 + aes(x='continent', y='country_count')    
 + geom_col(size=15)
 + geom_label(aes(label = 'country_count'),ha='center',color = "black",label_padding= 0.25)
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Top 50 Countries Continent wise Count")
 + labs(y = "Total No. of countries", x = "Continent")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.3),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white"),
    axis_text_x = element_text(color = "white", size = 10),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10}),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/7.png')

fig.figimage(image, xo=0, yo=0, alpha=0.2, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

Top 50 countries continent wise count:
> **22 European**, 
> **17 Asian**, 
> **6 South American**, 
> **4 North America**, 
> **1 African countries**.   

# **Least Fully Vaccinated People Per Hundred Country in a Continent**

In [ ]:
countries_pfvph = pd.DataFrame(vaccination_data.groupby("country")["people_fully_vaccinated_per_hundred"].max().sort_values(ascending = False))
countries_pfvph.reset_index(level=0,inplace=True)
countries_pfvph = convert_df_with_continent(countries_pfvph)
# bottom_20_countries_pfvph.groupby('continent', as_index=False).apply(lambda x: x.loc[x.people_fully_vaccinated_per_hundred.argmin(),])
bottom_20_countries_pfvph = pd.DataFrame(countries_pfvph.groupby("continent")["people_fully_vaccinated_per_hundred"].min())
bottom_20_countries_pfvph['country'] = ['']*bottom_20_countries_pfvph.shape[0]
bottom_20_countries_pfvph.reset_index(level=0,inplace=True)
for i in range(bottom_20_countries_pfvph.shape[0]):
    pvfh = bottom_20_countries_pfvph['people_fully_vaccinated_per_hundred'].iloc[i]
    continent = bottom_20_countries_pfvph['continent'].iloc[i]
    bottom_20_countries_pfvph['country'].iloc[i] = countries_pfvph.loc[(countries_pfvph['people_fully_vaccinated_per_hundred']==pvfh) & (countries_pfvph['continent']==continent)]['country'].values[0]
    if bottom_20_countries_pfvph['continent'].iloc[i] == 'NA':
        bottom_20_countries_pfvph['continent'].iloc[i] = 'North America'
    elif bottom_20_countries_pfvph['continent'].iloc[i] == 'SA':
        bottom_20_countries_pfvph['continent'].iloc[i] = 'South America'

In [ ]:
op.figure_size=(8,5)
ax = (ggplot(bottom_20_countries_pfvph)         
 + aes(x='continent', y='people_fully_vaccinated_per_hundred')    
 + geom_col(size=15)
 + geom_label(aes(label = 'country'),ha='center',color = "black",label_padding= 0.25)
 + scale_fill_manual(values = ["#B21236","#F03812","#FE8826","#FEB914","#2C9FA2","#002C2B","#F7E1C0"])
 + labs(title = "Countries with Least Fully Vaccinated People Per Hundred in a Continent")
 + labs(y = "", x = "Continent")
 + theme(
    panel_background = element_rect(fill = "black"),    
    plot_background = element_rect(fill = "black", color = "black"),
    legend_background = element_rect(fill = "black"),
    legend_key = element_blank(),
   
    panel_grid = element_line(size = 0.3),
    panel_grid_minor_y = element_blank(),
    panel_grid_major_y = element_blank(),
    
    legend_text = element_text(color = "white"),
    axis_text_x = element_text(color = "white", size = 10),
    axis_text_y = element_text(color = "white", size = 13, hjust = 1, margin={'b': 20, 't':10}),
    axis_title = element_text(color = "white", size = 14, hjust = 1),
    plot_title = element_text(color = "white", face = "bold", size = 16, hjust = 4, margin={'b': 20, 't':10}),
    panel_spacing_x = 1
  )
)

fig = ax.draw()

image = plt.imread('https://raw.githubusercontent.com/vineethbabu/coronavaccine_images/main/9.png')

fig.figimage(image, xo=0, yo=0, alpha=0.2, norm=None, cmap=None, vmin=None, vmax=None, origin=None, resize=True)

fig.show()

Continent wise countries with least fully vaccinated people: 
> * **Namibia** in Africa - **0.05**
> * **Ukraine** in Europe - **0.00**
> * **Kyrgyzstan** in Asia - **0.01**
> * **Guatemala** in North America - **0.01**
> * **Suriname** in South America - **0.01**

<center><h2><b>Thanks For Reading</b></h2></center>
<center><b>Please give your Suggestion and Comments.</b></center>